# Final Project: Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines - Hosted by DRIVENDATA

### 1. Problem Statement
*Predict the liklihood an individual received the H1N1 and flu vaccine*

### 2. Metrics and Assumptions
Previous submissions to *https://www.drivendata.org/competitions/66/flu-shot-learning/* suggest there is an estimated irreducible error of approximately 13 percent.

### 3. Data Source
The United States National Center for Immunized and Respiratory Diseases (NCIRD) and National Center for Health Statistics (NCHS), two organizations within the Centers for Disease Control and Prevention (CDC), conducted a national survey of American households to determine socio-economic indicators of individauls who have/have not received the H1N1 and seasonal flu vaccines. The survey was conducted between October 2009 and June 2010. 

### 4. Target and Feature Variables
This dataset has two target variables. Both are binary variables (0,1) that correspond to a respondent receiving either the H1N1 flu vaccine or the seasonal flu vaccine.

This dataset contains 36 columns (35 excluding the respondent id). 12 feature variables are objects containing economic, race, sex, age, and lifestyle indicators; and rest of the feature variables consist of binary and likert scale questions on health behaviors, indicators, and knowledge. I included all feature variables in my analysis.

### 5. Model Approach
Classification on probability. Performance for this model was evalutated using the Area Under the Receiver Operating Characteristic Curve (ROC AUC) for each target variable. Submission to DataDriven must include three columns: 'respondent_id', 'h1n1_vaccine', and 'seasonal_vaccine' where each target variable should be a probability between 0.0 and 1.0 that the respondent recieved each vaccine. 

#### Package Dependancies

In [127]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from catboost import CatBoostClassifier
from catboost import Pool, cv
import optuna

%matplotlib inline

### Data Import

In [128]:
df = pd.read_csv(r"Documents/H1N1 Predictions/training_set_features.csv", index_col='respondent_id')
labels_df = pd.read_csv(r"Documents/H1N1 Predictions/training_set_labels.csv", index_col='respondent_id')
test_df = pd.read_csv(r'Documents/H1N1 Predictions/test_set_features.csv', index_col='respondent_id')
np.testing.assert_array_equal(df.index.values, labels_df.index.values)

df = df.join(labels_df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 26615 non-null  float64
 1   h1n1_knowledge               26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_h1n1             24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker   

### Handling Missing Values

In [129]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [130]:
num_cols = df.select_dtypes('number').columns
obj_cols = df.select_dtypes('object').columns

test_num_cols = test_df.select_dtypes('number').columns

In [131]:
type(obj_cols)

pandas.core.indexes.base.Index

In [132]:
#TRAINING DATASET
for col in (obj_cols):
    df[col] = df[col].fillna(value=test_df[col].mode())
    
for col in num_cols:
    df[col] = df[col].fillna(value=-1)

#TEST DATASET
for col in (obj_cols):
    test_df[col] = test_df[col].fillna(value=test_df[col].mode())
    
for col in test_num_cols:
    test_df[col] = test_df[col].fillna(value=-1)

In [133]:
df = pd.get_dummies(df, columns=obj_cols)
test_df = pd.get_dummies(test_df, columns=obj_cols)

In [134]:
test_df.isna().sum()

h1n1_concern                      0
h1n1_knowledge                    0
behavioral_antiviral_meds         0
behavioral_avoidance              0
behavioral_face_mask              0
                                 ..
employment_occupation_vlluhbov    0
employment_occupation_xgwztkwe    0
employment_occupation_xqwwgdyp    0
employment_occupation_xtkaffoo    0
employment_occupation_xzmlyyjv    0
Length: 105, dtype: int64

In [135]:
# COPY DATASET TO PREDICT H1N1 VACCINE EXCLUSIVELY

h1n1_df = df
h1n1_df = h1n1_df.dropna(axis=1)
y = h1n1_df['h1n1_vaccine']
X = h1n1_df.drop(columns=['h1n1_vaccine','seasonal_vaccine'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [136]:
# IDENTIFY COLUMNS WITH CATEGORICAL FEATURES

cat_features = np.where(X_train.dtypes != float)[0]
cat_features

array([ 23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
        49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104], dtype=int64)

In [137]:
# API Reference: https://catboost.ai/docs/concepts/python-reference_pool.html
# CatBoost Pool preprocesses / wraps the dataset

train_dataset = Pool(data=X_train,
                     label=y_train,
                     cat_features = cat_features)

In [138]:
def objective(trial):
    
    # Optuna Trial API Reference: https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html
    # Trial determines best AUC mean by refining hyperparameter suggestions
    # For most param APIs: https://catboost.ai/docs/search/?query=od_type
    
    param = {
        # The maximum number of trees that can be built when solving machine learning problems.
        'iterations':trial.suggest_categorical('iterations', [100,200,300,500,1000,1200,1500,2000,3000,4000,5000]),
        
        # The learning rate. Used for reducing the gradient step.
        'learning_rate':trial.suggest_float("learning_rate", 0.001, 0.3),
        
        'random_strength':trial.suggest_int("random_strength", 1,10),
        
        # Defines the settings of the Bayesian bootstrap. It is used by default in classification and regression modes. 
        # Use the Bayesian bootstrap to assign random weights to objects.
        'bagging_temperature':trial.suggest_int("bagging_temperature", 0,10),
        
        'max_bin':trial.suggest_categorical('max_bin', [4,5,6,8,10,20,30]),
        
        # Defines how to perform greedy tree construction. ['SymmetricTree', 'Depthwise', 'Lossguide'] are all available hyperparams
        'grow_policy':trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        
        # The minimum number of training samples in a leaf. 
        'min_data_in_leaf':trial.suggest_int("min_data_in_leaf", 1,10),
        
        # Overfitting detector
        'od_type' : "Iter",
        
        # Number of iterations to continue the training after the iteration with the optimal metric value.
        'od_wait' : 100,
        
        # Depth of the tree
        "depth": trial.suggest_int("max_depth", 2,10),
        
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
        
        'one_hot_max_size':trial.suggest_categorical('one_hot_max_size', [5,10,12,100,500,1024]),
        
        'custom_metric' : ['AUC'],
        
        # Classificatoin Objectives and Metrics: https://catboost.ai/docs/concepts/loss-functions-classification.html
        # More about Log Loss: https://www.kaggle.com/dansbecker/what-is-log-loss
        "loss_function": 'Logloss',
        
        # Scaling
        'auto_class_weights':trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']),
        }
    
    # CatBoost CV API Reference: https://catboost.ai/docs/concepts/python-reference_cv.html
    # Utilizes OPTUNA params to determine best hyper paramaters and CatBoost's Cross Validation
  
    scores = cv(train_dataset,
            param,
            fold_count=10, 
            early_stopping_rounds=10,         
            plot=False, verbose=False)

    return scores['test-AUC-mean'].max()

In [139]:
# Tree-structure Parzen Estimator (TPE) Sampler API: https://optuna.readthedocs.io/en/stable/reference/samplers.html
# Creates reproducible results with sampler seed

sampler = optuna.samplers.TPESampler(seed=68)  
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=50)

[I 2021-04-07 22:36:14,722] A new study created in memory with name: no-name-d5438db2-09a7-481c-8c24-9c5ff3c26bf2
[I 2021-04-07 22:37:00,939] Trial 0 finished with value: 0.8659522576072736 and parameters: {'iterations': 300, 'learning_rate': 0.013964954297408176, 'random_strength': 1, 'bagging_temperature': 8, 'max_bin': 10, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 4, 'max_depth': 5, 'l2_leaf_reg': 21.328495943450676, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 0 with value: 0.8659522576072736.
[I 2021-04-07 22:37:26,619] Trial 1 finished with value: 0.8661515849000121 and parameters: {'iterations': 1200, 'learning_rate': 0.11477165079768124, 'random_strength': 9, 'bagging_temperature': 6, 'max_bin': 5, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 5, 'max_depth': 7, 'l2_leaf_reg': 0.5714362138520529, 'one_hot_max_size': 500, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 1 with value: 0.8661515849000121.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-04-07 22:37:39,261] Trial 2 finished with value: 0.8324344309562843 and parameters: {'iterations': 500, 'learning_rate': 0.10424841039250944, 'random_strength': 8, 'bagging_temperature': 3, 'max_bin': 6, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 3, 'max_depth': 10, 'l2_leaf_reg': 7.961547985302404e-08, 'one_hot_max_size': 5, 'auto_class_weights': 'Balanced'}. Best is trial 1 with value: 0.8661515849000121.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-04-07 22:38:14,891] Trial 3 finished with value: 0.8625857316074506 and parameters: {'iterations': 4000, 'learning_rate': 0.06981342303072555, 'random_strength': 6, 'bagging_temperature': 7, 'max_bin': 30, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 3, 'max_depth': 6, 'l2_leaf_reg': 1.253388039132331e-06, 'one_hot_max_size': 500, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 1 with value: 0.8661515849000121.


...

Stopped by overfitting detector  (10 iterations wait)


[I 2021-04-07 23:42:38,586] Trial 48 finished with value: 0.8672922791949803 and parameters: {'iterations': 3000, 'learning_rate': 0.25406816515011066, 'random_strength': 7, 'bagging_temperature': 10, 'max_bin': 5, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 7, 'max_depth': 6, 'l2_leaf_reg': 83.62197910519791, 'one_hot_max_size': 500, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 11 with value: 0.8707177746116492.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-04-07 23:43:24,139] Trial 49 finished with value: 0.8683587513592108 and parameters: {'iterations': 2000, 'learning_rate': 0.04294018055805611, 'random_strength': 5, 'bagging_temperature': 3, 'max_bin': 10, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 9, 'max_depth': 5, 'l2_leaf_reg': 3.116940448380019, 'one_hot_max_size': 1024, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 11 with value: 0.8707177746116492.


Stopped by overfitting detector  (10 iterations wait)


In [140]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}={},".format(key, value))

Number of finished trials: 50
Best trial:
  Value: 0.8707177746116492
  Params: 
    iterations=2000,
    learning_rate=0.01981954863467386,
    random_strength=1,
    bagging_temperature=8,
    max_bin=10,
    grow_policy=Lossguide,
    min_data_in_leaf=5,
    max_depth=7,
    l2_leaf_reg=78.38138081858969,
    one_hot_max_size=100,
    auto_class_weights=SqrtBalanced,


In [141]:
# CatBoostClassifier API Reference: https://catboost.ai/docs/concepts/python-reference_catboostclassifier.html

final_model = CatBoostClassifier(verbose=False,  cat_features=cat_features, 
                          **trial.params)

In [142]:
final_model.fit(X_train, y_train)
predictions_h1 = final_model.predict_proba(test_df)[:,1]
predictions_h1_train = final_model.predict_proba(X_test)[:,1]

In [143]:
roc_auc_score(y_test, predictions_h1_train)

0.8671530529635235

In [144]:
predictions_h1

array([0.16592446, 0.04416085, 0.22726992, ..., 0.23541126, 0.02632169,
       0.72728195])

In [145]:
seasonal_df = df
seasonal_df=seasonal_df.dropna(axis=1)
y = seasonal_df['seasonal_vaccine']
X = seasonal_df.drop(columns=['h1n1_vaccine','seasonal_vaccine'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [146]:
# Index number of cat features
cat_features = np.where(X_train.dtypes != float)[0]
cat_features

array([ 23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
        49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104], dtype=int64)

In [147]:
train_dataset = Pool(data=X_train,
                     label=y_train,
                     cat_features = cat_features)

In [148]:
def objective(trial):
    param = {
        'iterations':trial.suggest_categorical('iterations', [100,200,300,500,1000,1200,1500,2000,3000,4000,5000]),
        'learning_rate':trial.suggest_float("learning_rate", 0.001, 0.3),
        'random_strength':trial.suggest_int("random_strength", 1,10),
        'bagging_temperature':trial.suggest_int("bagging_temperature", 0,10),
        'max_bin':trial.suggest_categorical('max_bin', [4,5,6,8,10,20,30]),
        'grow_policy':trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'min_data_in_leaf':trial.suggest_int("min_data_in_leaf", 1,10),
        'od_type' : "Iter",
        'od_wait' : 100,
        "depth": trial.suggest_int("max_depth", 2,10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
        'one_hot_max_size':trial.suggest_categorical('one_hot_max_size', [5,10,12,100,500,1024]),
        'custom_metric' : ['AUC'],
        "loss_function": "Logloss",
        'auto_class_weights':trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']),
        }

    scores = cv(train_dataset,
            param,
            fold_count=10, 
            early_stopping_rounds=10,         
            plot=False, verbose=False)

    return scores['test-AUC-mean'].max()

In [150]:
sampler = optuna.samplers.TPESampler(seed=68)  # Make the sampler behave in a deterministic way.
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=50)

[I 2021-04-08 06:45:42,423] A new study created in memory with name: no-name-56004c1d-182d-4a61-b9f8-c199e3f637d5
[I 2021-04-08 06:46:34,734] Trial 0 finished with value: 0.8585526376329218 and parameters: {'iterations': 300, 'learning_rate': 0.013964954297408176, 'random_strength': 1, 'bagging_temperature': 8, 'max_bin': 10, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 4, 'max_depth': 5, 'l2_leaf_reg': 21.328495943450676, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 0 with value: 0.8585526376329218.
[I 2021-04-08 06:47:05,679] Trial 1 finished with value: 0.8603229456814425 and parameters: {'iterations': 1200, 'learning_rate': 0.11477165079768124, 'random_strength': 9, 'bagging_temperature': 6, 'max_bin': 5, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 5, 'max_depth': 7, 'l2_leaf_reg': 0.5714362138520529, 'one_hot_max_size': 500, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 1 with value: 0.8603229456814425.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-04-08 06:47:25,654] Trial 2 finished with value: 0.8368923253294923 and parameters: {'iterations': 500, 'learning_rate': 0.10424841039250944, 'random_strength': 8, 'bagging_temperature': 3, 'max_bin': 6, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 3, 'max_depth': 10, 'l2_leaf_reg': 7.961547985302404e-08, 'one_hot_max_size': 5, 'auto_class_weights': 'Balanced'}. Best is trial 1 with value: 0.8603229456814425.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-04-08 06:48:06,981] Trial 3 finished with value: 0.8575016417253865 and parameters: {'iterations': 4000, 'learning_rate': 0.06981342303072555, 'random_strength': 6, 'bagging_temperature': 7, 'max_bin': 30, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 3, 'max_depth': 6, 'l2_leaf_reg': 1.253388039132331e-06, 'one_hot_max_size': 500, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 1 with value: 0.8603229456814425.


...

Stopped by overfitting detector  (10 iterations wait)


[I 2021-04-08 07:48:35,562] Trial 46 finished with value: 0.8624956015685921 and parameters: {'iterations': 2000, 'learning_rate': 0.007736085095600653, 'random_strength': 3, 'bagging_temperature': 4, 'max_bin': 4, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 7, 'max_depth': 8, 'l2_leaf_reg': 7.499443521533301, 'one_hot_max_size': 100, 'auto_class_weights': 'Balanced'}. Best is trial 44 with value: 0.86263419571034.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-04-08 07:48:51,968] Trial 47 finished with value: 0.8478031954404605 and parameters: {'iterations': 100, 'learning_rate': 0.029734414414858742, 'random_strength': 5, 'bagging_temperature': 4, 'max_bin': 4, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 6, 'max_depth': 9, 'l2_leaf_reg': 1.0521712916014843, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 44 with value: 0.86263419571034.
[I 2021-04-08 07:49:42,028] Trial 48 finished with value: 0.8396113618392009 and parameters: {'iterations': 300, 'learning_rate': 0.004064222944990553, 'random_strength': 3, 'bagging_temperature': 3, 'max_bin': 4, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 5, 'max_depth': 10, 'l2_leaf_reg': 0.24571187865982283, 'one_hot_max_size': 100, 'auto_class_weights': 'Balanced'}. Best is trial 44 with value: 0.86263419571034.
[I 2021-04-08 07:51:03,793] Trial 49 finished with value: 0.8620568529725521 and parameters: {'iterations': 1000, 'learning_rate': 0.033572126975507

Stopped by overfitting detector  (10 iterations wait)


In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}={},".format(key, value))

In [ ]:
final_model_seasonal = CatBoostClassifier(verbose=False,  cat_features=cat_features, 
                          **trial.params)

In [ ]:
final_model_seasonal.fit(X_train, y_train)
predictions_seasonal = final_model_seasonal.predict_proba(test_df)[:,1]
predictions_seasonal_train = final_model_seasonal.predict_proba(X_test)[:,1]

In [ ]:
predictions_seasonal

In [ ]:
roc_auc_score(y_test, predictions_seasonal_train)

In [ ]:
seasonal = pd.Series(predictions_seasonal)

In [ ]:
h1n1 = pd.Series(predictions_h1)

In [ ]:
h1n1=h1n1.reset_index().drop('index',axis=1)
seasonal=seasonal.reset_index().drop('index',axis=1)

In [ ]:
respondent_id_df = pd.DataFrame(
    test_df.index
)

In [ ]:
submission = respondent_id_df.merge(h1n1 ,how='outer', left_index=True,right_index=True)
submission = submission.merge(seasonal ,how='outer', left_index=True,right_index=True)
submission = submission.set_index('respondent_id')
submission = submission.rename(columns={'0_x':'h1n1_vaccine','0_y':'seasonal_vaccine'})

In [ ]:
submission.to_csv(r"C:\Users\Horri\Downloads\submission_20210331v7.csv")

In [ ]:
submission